In [1]:
# 보스턴 집값 데이터를 가져옵시다. 
from keras.datasets import boston_housing
(train_data, train_labels),(test_data, test_labels) = boston_housing.load_data()

In [2]:
print(train_data.shape)
print(test_data.shape)
# 13개의 특성이 있고 데이터의 수가 매우 적습니다.

(404, 13)
(102, 13)


In [3]:
print(train_labels[0])
print(train_data[0])  # 데이터의 수치가 들쭉날쭉한 걸 보니 스케일링을 통해 수치를 맞춰주는 작업이 필요할 것 같네요.

15.2
[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [4]:
# 스케일링으로 데이터 전처리를 해봅시다.
mean = train_data.mean(axis=0)  # 각 컬럼 별로 평균을 구합니다. 가로가 0, 세로가 1입니다.
train_data -= mean  # 학습데이터에서 평균을 낸 값을 뺍니다. train_data = train_data - mean과 같은 표현입니다.
std = train_data.std(axis=0)  # 표준편차를 각 컬럼 별로 구해봅시다.
train_data /= std  # 학습데이터를 표준편차로 나눠봅시다.

train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [5]:
# 신경망을 제작해봅시다. 신경망 제작은 모델구성과 같은 말입니다.
from keras import models
from keras import layers

# model = models.Sequential()
# model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1])))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(1))
# model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# K겹 또는 K폴더 검증이란, 데이터가 얼마 없을 때 데이터를 3분할 해서 처음엔 검증, 훈련, 훈련 두번째엔 훈련, 검증, 훈련 세번째엔 훈련, 훈련, 검증 식으로 모든 데이터를 교차로 싹 도는 방법을 의미합니다.
# 근데 이 경우, 반복해서 데이터를 초기화해야하는 번거로움이 있기 때문에 데이터를 초기화 해서 계속 검증을 돌리는 함수를 만들어 봅시다.
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop',
                loss='mse',
                metrics=['mse'])
  return model

In [8]:
# 함수를 만들었으니 이제 K-folder 검증을 사용해 훈련 검증을 해봅시다.
import numpy as np

k=4  # folder의 갯수를 4로 지정합니다.
num_val_samples = len(train_data)//k  # 샘플의 갯수(한 폴더의 데이터 수)는 101개씩 4덩이로 쪼개지면 되겠네요.
all_scores = []

for i in range(k):  # K의 숫자만큼 반복하게 됩니다.
    print('처리중인 폴더 #', i)
    
    # 검증데이터를 앞에서 구한 샘플의 갯수만큼 잘라봅시다.
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_labels = train_labels[i * num_val_samples: (i + 1) * num_val_samples]
    
    # 검증데이터를 제외하고 나머지는 전부 학습데이터로 쓰면 됩니다. 3조각으로 나눠진 데이터를 합치려면 어떻게 해야할까요?
    data1 = train_data[:i * num_val_samples]
    # print(0,i * num_val_samples)
    data2 = train_data[(i+1) * num_val_samples :]
    # print((i+1) * num_val_samples, 404)
    # 라벨도 합쳐줍시다.
    data1_labels = train_labels[:i * num_val_samples]
    data2_labels = train_labels[(i+1) * num_val_samples :]

    partial_train_data = np.concatenate([data1, data2], axis=0)
    partial_train_labels = np.concatenate([data1_labels, data2_labels], axis=0)
    
    # 이제 학습시켜 봅시다.
    model = build_model()
    model.summary()
    
    model.fit(partial_train_data, partial_train_labels, epochs=500, batch_size=128, verbose=0)  # verbose는 훈련상황을 보여줄거냐 말거냐 입니다.
    
    # 그리고 모델을 검증해봅시다.
    val_mse, val_mae = model.evaluate(val_data, val_labels)
    print(val_mse, val_mae)
    all_scores.append(val_mae)  # 아까 만들어 놓은 리스트 객체에 누적추가할 수 있도록 합시다.

처리중인 폴더 # 0
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                896       
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 2ms/step - loss: 8.0118 - mse: 8.0118
8.01176643371582 8.01176643371582
처리중인 폴더 # 1
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)       

In [7]:
np.mean(all_scores)

11.956292152404785